Data imputation

In [102]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data_clean.csv', sep=',', encoding='latin-1')
print(df.head())
print(df.columns)
print(df.shape)


    SEQN  RIDAGEYR  RIAGENDR  RIDRETH1  RIDRETH3  DMDCITZN  DMDEDUC2  MIALANG  \
0  73557        69         1         4         4       1.0       3.0      1.0   
1  73558        54         1         3         3       1.0       3.0      1.0   
2  73559        72         1         3         3       1.0       4.0      1.0   
3  73561        73         2         3         3       1.0       5.0      1.0   
4  73562        56         1         1         1       1.0       4.0      1.0   

   DMDHHSIZ  INDHHIN2  ...  PAQ635  PAQ650  PAQ665  PAD680  PAQ706  PAQ710  \
0         3       4.0  ...       2       2       2   600.0     NaN     2.0   
1         4       7.0  ...       2       2       2   540.0     NaN     4.0   
2         2      10.0  ...       2       2       1   300.0     NaN     4.0   
3         2      15.0  ...       2       2       2   480.0     NaN     1.0   
4         1       9.0  ...       2       2       2   360.0     NaN     5.0   

   LBXTC  highLDL  highbp        bmi  
0  16

Data imputation for 3 columns

In [103]:
df = df[["BPXPLS", "BMXARMC", "bmi", "highbp" ]]
# df["highbp"].isnull().sum()
# df["highbp"]


Convert dataframe to numpy array

In [104]:
# df[df.isnull()] = 0
df.shape
np_array = df.to_numpy(copy=True, na_value=np.nan)

# for column in df.columns:
#     df_null_inx.append(df[df[column].isnull()].index.tolist())

# # df_null_inx

# for column in df.columns:
#     df[column].fillna((df[column].mean()), inplace=True)


Get non-missing indices

In [105]:
# np_array
# nonmissing_indices = np.flatnonzero(np_array > 0.1)
# nonmissing_indices = np_array.flat[np.argwhere(np.isfinite(np_array))]
# nonmissing_indices = np_array.flat[np.isfinite(np_array.flat)]
# nonmissing_indices = np.flatnonzero(~ np.isnan(np_array))
nonmissing_indices = np.argwhere(~ np.isnan(np_array.flatten())).reshape(-1)
missing_indices = np.argwhere(np.isnan(np_array.flatten())).reshape(-1)
# nonmissing_indices.reshape(-1)
print(nonmissing_indices)
print(len(nonmissing_indices))
print(len(missing_indices))

# u, s, v = np.linalg.svd(df)


[    0     1     2 ... 23073 23074 23075]
21363
1713


fill an array with available data

In [106]:
rating_matrix_ini = np.zeros(np_array.shape)
rating_matrix_ini.flat[nonmissing_indices] = np_array.flat[nonmissing_indices]


# rating_matrix_ini
# for column in np_array.shape[1]:
#     np_array[:,column] = 

Impute missing data

In [107]:
np.isnan(np_array)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [ True, False, False,  True],
       [False, False, False, False],
       [False, False, False, False]])

In [108]:
# rating_matrix_ini[6,:]
# for column in range(np_array.shape[1]):
#     np_array[np.where(np_array[:,column] == np.nan),column] = np.mean(np_array[:,column])
col_mean = np.nanmean(np_array, axis=0)
inds = np.where(np.isnan(np_array))
np_array[inds] = np.take(col_mean, inds[1])




In [109]:
np_array.shape
np.isnan(np_array)

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

Fit low rank model for rank 1

In [110]:
def fit_low_rank_model(rank,rating_matrix_ini,train_ind,train_data,n_iter,convergence_thresh,verbose):
    """Fit the low rank model. 
    Return the estimation of the low rank model - (n_movies * n_users) matrix

    Keyword arguments:
    rank -- the rank of low rank model
    rating_matrix_ini -- imputed initialization
    train_ind -- index of training data
    train_data -- ratings of training set
    n_iter -- the max number of iterations
    convergence_thresh -- the threshold of convergence to 0
    """
    previous_fitting_error = 100
    # Initialization
    low_rank_estimate = np.zeros(rating_matrix_ini.shape)
    # fill input data
    low_rank_estimate.flat[train_ind] = train_data
    # get the indexes of missing data
    missing_inds = np.where(low_rank_estimate.flat == 0)
    # fill missing data with imputed values
    low_rank_estimate.flat[missing_inds] = rating_matrix_ini.flat[missing_inds]

    
    for ind in range(n_iter):
        # Updates
        low_rank_estimate.flat[train_ind] = train_data
        u, s, v = np.linalg.svd(low_rank_estimate)
        s_matrix = s  * np.eye(len(s))
        low_rank_estimate = np.matmul(np.matmul(u[:,0:rank], s_matrix[0:rank,0:rank]), v[0:rank,:])
        # Compute error
        fitting_error = np.sqrt(((train_data - low_rank_estimate.flat[train_ind])**2).mean())
        if verbose:
            print("Iteration " + str(ind) + " Error: " + str(fitting_error))
        
        # Stopping criterion
        if (fitting_error <= convergence_thresh):
            print('converged, breaking')
            break
    return low_rank_estimate

n_iter = 5
convergence_thresh = 1e-4
verbose = True
rank = 1
train_data = np_array.flat[nonmissing_indices] 
estimate =fit_low_rank_model(rank,rating_matrix_ini,nonmissing_indices,
        train_data,n_iter,convergence_thresh,verbose)
print(estimate)

Iteration 0 Error: 5.688635605794431
Iteration 1 Error: 5.176434783248822
Iteration 2 Error: 4.974143717879422
Iteration 3 Error: 4.862420292030783
Iteration 4 Error: 4.797935047569216
[[82.42372315 37.4323088  33.16084306  0.40901999]
 [74.03016103 33.62041585 29.78393183  0.36736773]
 [69.33358835 31.48749159 27.89439926  0.34406143]
 ...
 [53.37316846 24.23914921 21.47317781  0.26485934]
 [60.9758548  27.69187    24.53190264  0.30258696]
 [80.76200102 36.67764626 32.49229637  0.40077385]]
